In [8]:
!python3 -V
!pip install --ignore-installed blinker
!pip install Flask

Python 3.11.0rc1
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import subprocess
import requests
import os

In [2]:

flask_subdir = 'yolo_flask'

#image_path = '716original_2024-02-27-150444_DeepBlue_20ngmL-1.jpg'
#crop_image_path = '716original_2024-02-27-150444_DeepBlue_20ngmL-1_crop.jpg'

image_path = '738original_2024-02-27-151112_DeepBlue_100ngmL-1.jpg'
crop_image_path = '738original_2024-02-27-151112_DeepBlue_100ngmL-1_crop.jpg'


#image_path = os.environ["IMAGE_NAME"]
#crop_image_path = os.environ["IMAGE_NAME_CROPPED"]
#result_info = os.environ["RESULT"]

In [3]:
def is_service_running(service_name):
    try:
        # Change to the 'yolo_flask' directory
        os.chdir(flask_subdir)
        
        # Check if the service is running
        result = subprocess.run(
            ["docker", "compose", "ps", "--services", "--filter", "status=running"],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE
        )
        # Return to the original directory
        os.chdir('..')
        
        # If the service is running, it will appear in the output
        return service_name in result.stdout.decode('utf-8')
    except subprocess.CalledProcessError as e:
        print(f"Error checking service status: {e}")
        return False



def start_service(service_name):
    try:
        # Change to the 'yolo_flask' directory
        os.chdir(flask_subdir)
        
        # Start the service using docker-compose
        subprocess.run(
            ["docker", "compose", "up", "-d", service_name],
            check=True
        )
        
        # Return to the original directory
        os.chdir('..')
        
    except subprocess.CalledProcessError as e:
        print(f"Error starting service: {e}")

In [4]:
def send_inference_request(data):
    url = "http://localhost:5000/predict"  # Adjust the URL to match your Flask route
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()
        return response.json()  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"Error sending inference request: {e}")
        return None

In [5]:
print(os.listdir() )

['738original_2024-02-27-151112_DeepBlue_100ngmL-1_crop.jpg', 'run_jupyter.ipynb', '716original_2024-02-27-150444_DeepBlue_20ngmL-1_crop.jpg', 'Install_list.ipynb', '.ipynb_checkpoints', '__pycache__', 'yolo_flask']


In [6]:
service_name = "yolo_flask"  # Replace with your service name in docker-compose.yml
data = {"image_path": '/716original_2024-02-27-150444_DeepBlue_20ngmL-1.jpg',
       "crop_image_path":'/cropped/716original_2024-02-27-150444_DeepBlue_20ngmL-1_crop.jpg',
       "result_info": "/cropped/result_info.txt"}  # Replace with your actual data


if not is_service_running(service_name):
    print(f"Service {service_name} is not running. Starting service...")
    start_service(service_name)
else:
    print(f"Service {service_name} is already running.")


# Send the inference request
result = send_inference_request(data)
if result:
    print(f"Inference result: {result}")
else:
    print("Failed to get inference result.")

FileNotFoundError: [Errno 2] No such file or directory: 'docker'